# 11 시계열

## 11.1 날짜, 시간 자료형, 도구

datetime 모듈을 활용한 데이터 분석

In [1]:
from datetime import datetime

In [2]:
now = datetime.now()

In [3]:
now

datetime.datetime(2021, 5, 7, 1, 17, 10, 230958)

In [4]:
now.year, now.month, now.day

(2021, 5, 7)

In [15]:
delta = datetime(2021, 5, 7) - datetime(2021, 5, 1, 8, 14)
delta.days, delta.seconds

(5, 56760)

In [16]:
delta = datetime(2021, 5, 7) - datetime(2021, 5, 1)
delta.days, delta.seconds

(6, 0)

In [29]:
delta / 2 + delta * 4

datetime.timedelta(days=27)

In [17]:
from datetime import timedelta

In [19]:
start = datetime(2021, 5, 1)

In [23]:
start + timedelta(12), start + 2 * timedelta(12)

(datetime.datetime(2021, 5, 13, 0, 0), datetime.datetime(2021, 5, 25, 0, 0))

In [22]:
start - delta

datetime.datetime(2021, 4, 25, 0, 0)

<center>datetime 모듈의 자료형</center>    

|자료형|설명|  
|------|---|  
|date|날짜(연,월,일)을 저장|  
|time|하루의 시간을 시,분,초,마이크로초 단위 저장|  
|datetime|날짜와 시간을 저장|  
|timedelta|datetime값 간의 차이(일,초,마이크로초) 표현|   

### 11.1.1 문자열을 datetime으로 변환하기

datetime객체와 pandas Timestamp 객체는 str나 strftime로 문자열을 포맷 규칙을 넘겨서 서로 나타낼 수 있다.

In [31]:
stamp = datetime(2011, 1, 3)

In [34]:
str(stamp), stamp.strftime('%Y년 %m월 %d일')

('2011-01-03 00:00:00', '2011년 01월 03일')

Datetime 포맷 규칙 참고: [Python Datetime Doc](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

In [40]:
value1 = '2021-05-07'
value2 = '2021/5/7'
# 다음과 같이 포맷 규칙을 사용하여 문자열을 datetime 객체로 변환할 수 있다.
datetime.strptime(value1, '%Y-%m-%d'), datetime.strptime(value2, '%Y/%m/%d')

(datetime.datetime(2021, 5, 7, 0, 0), datetime.datetime(2021, 5, 7, 0, 0))

In [44]:
from dateutil.parser import parse

# 사실 위의 모듈을 사용하면 포맷 설정없이 편하게 변환할 수 있다.
parse(value1), parse(value2)

(datetime.datetime(2021, 5, 7, 0, 0), datetime.datetime(2021, 5, 7, 0, 0))

In [46]:
# 미국식? 날짜 표기의 경우
value3 = '7/5/2021'
parse(value3, dayfirst=True)

datetime.datetime(2021, 5, 7, 0, 0)

In [50]:
# 판다스 to_datatime을 이용하면 문자열 배열을 한꺼번에 변환 가능하다.
import pandas as pd
pd.to_datetime([value1, value2])

DatetimeIndex(['2021-05-07', '2021-05-07'], dtype='datetime64[ns]', freq=None)

## 11.2 시계열 기초

In [102]:
dates = pd.period_range(start='2021/4/28', end='2021/5/8', periods=None, freq='D')
dates

PeriodIndex(['2021-04-28', '2021-04-29', '2021-04-30', '2021-05-01',
             '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05',
             '2021-05-06', '2021-05-07', '2021-05-08'],
            dtype='period[D]', freq='D')

In [103]:
pd.period_range(end='2021/5/8', periods=11, freq='D')

PeriodIndex(['2021-04-28', '2021-04-29', '2021-04-30', '2021-05-01',
             '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05',
             '2021-05-06', '2021-05-07', '2021-05-08'],
            dtype='period[D]', freq='D')

In [104]:
pd.period_range(start='2021/4/28', periods=11, freq='D')

PeriodIndex(['2021-04-28', '2021-04-29', '2021-04-30', '2021-05-01',
             '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05',
             '2021-05-06', '2021-05-07', '2021-05-08'],
            dtype='period[D]', freq='D')

In [105]:
import numpy as np
ts = pd.Series(np.random.randn(len(dates)), index=dates)
ts

2021-04-28   -0.576577
2021-04-29    0.980879
2021-04-30   -0.889797
2021-05-01    1.441594
2021-05-02    0.463585
2021-05-03    0.096873
2021-05-04    0.087057
2021-05-05    0.365575
2021-05-06    1.976146
2021-05-07    1.425787
2021-05-08   -1.381235
Freq: D, dtype: float64

In [106]:
ts.index

PeriodIndex(['2021-04-28', '2021-04-29', '2021-04-30', '2021-05-01',
             '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05',
             '2021-05-06', '2021-05-07', '2021-05-08'],
            dtype='period[D]', freq='D')

In [107]:
ts + ts[::2]

2021-04-28   -1.153154
2021-04-29         NaN
2021-04-30   -1.779595
2021-05-01         NaN
2021-05-02    0.927170
2021-05-03         NaN
2021-05-04    0.174114
2021-05-05         NaN
2021-05-06    3.952292
2021-05-07         NaN
2021-05-08   -2.762470
Freq: D, dtype: float64

### 11.2.1 색인, 선택, 부분 선택

In [108]:
stamp = ts.index[4]
stamp, ts[stamp]

(Period('2021-05-02', 'D'), 0.4635849836673764)

In [109]:
# 다음과 같이 문자열만으로도 날짜 인덱스에 맞게 자동 변환되어 값을 불러올 수 있다.
ts['20210428'],ts['2021/04/28'],ts['2021-04-28']

(-0.5765772482127955, -0.5765772482127955, -0.5765772482127955)

In [110]:
# 해당 월의 Series를 불러올 수 있다. 하지만, '-'나 '/'같은 포맷이 필요하다.
ts['2021-04'],ts['2021/04'] #,ts['202104']

(2021-04-28   -0.576577
 2021-04-29    0.980879
 2021-04-30   -0.889797
 Freq: D, dtype: float64,
 2021-04-28   -0.576577
 2021-04-29    0.980879
 2021-04-30   -0.889797
 Freq: D, dtype: float64)

In [111]:
ts['20210430':]

2021-04-30   -0.889797
2021-05-01    1.441594
2021-05-02    0.463585
2021-05-03    0.096873
2021-05-04    0.087057
2021-05-05    0.365575
2021-05-06    1.976146
2021-05-07    1.425787
2021-05-08   -1.381235
Freq: D, dtype: float64